# Lab2 - EA for Set Covering Problem

# Libraries

In [3]:
import numpy as np
import logging
from collections import namedtuple
import random
from matplotlib import pyplot as plt
import time

# Code

## Problem instance generator

In [4]:
def problem(N: int, seed=42):
    """Creates an instance of the problem"""

    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

## Evolutionary Algorithm

### Fitness

- The fitness evaluate how fit a given solution is in solving the set covering problem. Set covering problem looks to minimize the size of the final solution to get all number in range(N).

In [5]:
def fitness(genome: list,N: int):
    """ Definition of the fitness"""
    return sum(len(_) for _ in genome)

### Tournament

The set covering problem is a minimisation problem, therefore we will try to minimize it. As we are looking to the values closest to zero, we can use the absolute value of the fitness

In [6]:
def tournament(population: list, tournament_size=2):
    """Tournament function"""
    return min(random.choices(population, k=tournament_size), key=lambda i: abs(i.fitness))

### Cross-Over

As seen in cass, I take a random point of my genome and then I cut at this point. The cut must be maximum the length of the smallest genome

In [7]:
def cross_over(g1: list, g2: list):
    cut = random.randint(1, min([len(g1),len(g2)]))
    return g1[:cut] + g2[cut:]

### Mutation

Three types of mutation:
- if len(genome) < Len(goal) ==> insertion of random array from the initial set
- if len(genome) > len(goal) ==> deletion of a random array
- if len(genome) == len(goal) ==> change random value with one from the initial set

In [8]:
def mutation(g: list,goal: set, initial_set:list):
    if sum(len(_) for _ in g) > len(goal):
        point = random.randint(0, len(g)-1)
        del g[point]
    elif sum(len(_) for _ in g) < len(goal):
        point = random.randint(0, len(initial_set)-1)
        g.append(initial_set[point])
    else:
        point1 = random.randint(0, len(g)-1)
        point2 = random.randint(0, len(initial_set)-1)
        del g[point1]
        g.append(initial_set[point2])
    return g

### Genetic Algorithm

In [9]:
logging.getLogger().setLevel(logging.INFO)

#### Initial Population

##### Infeasible Solutions
- Our problem provide us a lot of an infeasible solution therefore it is better to take this values. 
- As the heuristic repair could be dangerous, I will not use it
- We will give a penalty to them. As set covering problem is a minminization problem, we will try to maximize this values
- According, we will provide a factor of 100 * N.

In [10]:
def feasible(solution, GOAL):
    return set([el for array in solution for el in array]) == GOAL

In [11]:
def penalty(value,N):
    return value*N*100

##### Function

In [31]:
Individual = namedtuple("Individual", ["genome", "fitness"])
def initial_population(N: int,POP_SIZE: int,seed = 42):
    """ Initialisation of the population according to feasibility"""
    population = list()
    initial_set = problem(N,seed)
    GOAL = set(list(range(N)))
    for values in range(POP_SIZE):
       genome = random.sample(initial_set,random.randint(1,N))# The ideal solution is N array of length 1 with all value
       if feasible(genome,GOAL):
           population.append(Individual(genome,fitness(genome,N)))
       else:
           population.append(Individual(genome, penalty(fitness(genome,N),N)))
    logging.info(f"init: N={N} pop_size={len(population)}; min_bloat_init={min(population, key=lambda i: abs(i.fitness))[1]}")
    return population,initial_set

#### Performance Evolution

In [13]:
def plot_result(fitness_log,NUM_GENERATIONS ):
    off_line = [max(f[1] for f in fitness_log if f[0] == x) / (x + 1) for x in range(NUM_GENERATIONS)]
    on_line = [max(f[1] for f in fitness_log if f[0] <= x) / (x + 1) for x in range(NUM_GENERATIONS)]
    gen_best = [max(f[1] for f in fitness_log if f[0] == x) for x in range(NUM_GENERATIONS)]

    plt.figure(figsize=(15, 6))
    #plt.scatter([x for x, _ in fitness_log], [y for _, y in fitness_log], marker=".")
    plt.plot([x for x, _ in enumerate(gen_best)], [y for _, y in enumerate(gen_best)])
    plt.plot([x for x, _ in enumerate(on_line)], [y for _, y in enumerate(on_line)])
    plt.plot([x for x, _ in enumerate(off_line)], [y for _, y in enumerate(off_line)])



#### Evolution

In [27]:
fitness_log = []
def evolution(N: int,population: list,POPULATION_SIZE: int, NUM_GENERATIONS:int, OFFSPRING_SIZE:int,initial_set:list):
    fitness_log = [(0, i.fitness) for i in population]
    GOAL = set(list(range(N)))
    for g in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            if random.random() < 0.3:
                p = tournament(population)
                o = mutation(p.genome,GOAL,initial_set)
            else:
                p1 = tournament(population)
                p2 = tournament(population)
                o = cross_over(p1.genome, p2.genome)
            if feasible(o,GOAL):
                f = fitness(o,N)
            else:
                f = penalty(fitness(o,N),N)
            fitness_log.append((g+1, f))
            offspring.append(Individual(o, f))
        population += offspring
        population = sorted(population, key=lambda i: abs(i.fitness))[:POPULATION_SIZE]
    #plot_result(fitness_log,10*N)
    return population

In [28]:
def algorithm(N,pop_size,num_gen,offsize):
    POPULATION_SIZE = pop_size
    OFFSPRING_SIZE = offsize
    NUM_GENERATIONS = num_gen
    ### Initialisation of the problem
    population,initial_set = initial_population(N,POPULATION_SIZE)
    ### Evolution
    return evolution(N,population,POPULATION_SIZE,NUM_GENERATIONS,OFFSPRING_SIZE,initial_set)

## 5 TO 100

In [43]:
tic = time.perf_counter()
for N in [5,10,20,50,100]:
    results = algorithm(N,N*N,100*N,N*10)[:3]
    logging.info(f'Top 3 bests results for {N}: {[el.fitness for el in results]}')
    toc = time.perf_counter()
    logging.info(f'Found in {toc-tic:0.4f} s')

INFO:root:init: N=5 pop_size=25; min_bloat_init=5
INFO:root:Top 3 bests results for 5: [5, 5, 5]
INFO:root:Found in 0.1651 s
INFO:root:init: N=10 pop_size=100; min_bloat_init=19
INFO:root:Top 3 bests results for 10: [10, 10, 10]
INFO:root:Found in 0.7967 s
INFO:root:init: N=20 pop_size=400; min_bloat_init=35
INFO:root:Top 3 bests results for 20: [23, 24, 24]
INFO:root:Found in 3.5485 s
INFO:root:init: N=50 pop_size=2500; min_bloat_init=121
INFO:root:Top 3 bests results for 50: [66, 66, 66]
INFO:root:Found in 25.9789 s
INFO:root:init: N=100 pop_size=10000; min_bloat_init=260
INFO:root:Top 3 bests results for 100: [176, 183, 183]
INFO:root:Found in 149.6102 s


## 500 and 1000

After all test, I observe that for a initial population > 5000 there is no more improvement.
And After a offsize > 1500 my performances were decreasing

In [47]:
tic = time.perf_counter()
for N in [500,1000]:
    results = algorithm(N,5000,1000,1200)[:3]
    logging.info(f'Top 3 bests results for {N}: {[el.fitness for el in results]}')
    logging.info(f'Found in {time.perf_counter()-tic:0.4f} s')

INFO:root:init: N=500 pop_size=5000; min_bloat_init=2094
INFO:root:Top 3 bests results for 500: [1401, 1401, 1401]
INFO:root:Found in 55.9269 s
INFO:root:init: N=1000 pop_size=5000; min_bloat_init=5307
INFO:root:Top 3 bests results for 1000: [3558, 3603, 3605]
INFO:root:Found in 222.6616 s
